# Class 8

- Review of hypothesis testing

- Introduction to linear regression

# Bootstrap 

A data scientist is using the data in a random sample to estimate an unknown statistical parameter. She uses the sample to calculate the value of a statistic that she will use as her estimate.

Once she has calculated the observed value of her statistic, she could just present it as her estimate and go on her merry way. But she’s a data scientist. She knows that her random sample is just one of numerous possible random samples, and thus her estimate is just one of numerous plausible estimates.

By how much could those estimates vary? To answer this, it appears as though she needs to draw another sample from the population, and compute a new estimate based on the new sample. But she doesn’t have the resources to go back to the population and draw another sample.

It looks as though the data scientist is stuck.

Fortunately, a brilliant idea called the bootstrap can help her out. Since it is not feasible to generate new samples from the population, the bootstrap generates new random samples by a method called resampling: the new samples are drawn at random from the original sample.

- We observed before that a large random sample from a population resembles the population from which it was drawn.

- Consider gestation in days from the pantheria dataset.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns


time_use_df = pd.read_csv('gss_tu2016_main_file.csv')

```
dur01              Duration - Sleeping, resting, relaxing, sick in bed

           VALUE  LABEL
               0  No time spent doing this activity
            9996  Valid skip
            9997  Don't know
            9998  Refusal
            9999  Not stated

           Data type: numeric
           Missing-data codes: 9996-9999
           Record/columns: 1/65-68
```

```
chh0014c           Child(ren) in household - 0 to 14 years

           VALUE  LABEL
               0  None
               1  One
               2  Two
               3  Three or more
               6  Valid skip
               7  Don't know
               8  Refusal
               9  Not stated

```

In [ ]:
time_use_df['dur01'].plot.hist(color = 'grey', edgecolor = 'black');

In [ ]:
time_use_df['dur01'].median()

- Draw a random sample with replacement of the same size (number of rows) as the original sample.

- This opens the possibility for the new sample to be different from the original sample.

In [ ]:
gest_bsample = time_use_df['dur01'].sample(frac = 1, replace=True)

gest_bsample.plot.hist(color = 'grey', edgecolor = 'black');

In [ ]:
gest_bsample.mean()

## Why is this a good idea?

- By the law of averages, the distribution of the original sample is likely to resemble the population.  

- The distributions of all the “resamples” are likely to resemble the original sample. 

- So the distributions of all the resamples are likely to resemble the population as well.

## Resampled mean

- We generated one resampled or *bootstrapped* median above.

- By resampling many times we can compute the empirical distribution of the median gestation time.

In [ ]:
def one_bs_mean():
    gest_med_bsample = time_use_df['dur01'].sample(frac = 1, replace=True).mean()
    return gest_med_bsample

In [ ]:
one_bs_mean()

- Now let's compute many bootstrap medians by writing a `for` loop.

In [ ]:
bootstrap_means = []  # empty list to collect medians

for _ in range(5000):
    bootstrap_means.append(one_bs_mean())

In [ ]:
plt.hist(bootstrap_means, color = 'grey', edgecolor = 'black');

## Boostrap confidence intervals

- We can use the bootstrap distribution to construct a range of values such that 95% of the random samples will contain the true median.

- The range of values is called a confidence interval.

- A 95% confidence interval for the median can be constructed finding the 2.5% percentile and the 97.5% percentile.  

- The reason for choosing 2.5% and 97.5% is that 0.05/2 = 0.025, and 1 - 0.025 = 0.975.   

- We can do this using the `percentile` function in `numpy`.

In [ ]:
np.percentile(bootstrap_means, 2.5)

In [ ]:
np.percentile(bootstrap_means, 97.5)

A 95% bootstrap confidence interval for mean is 520.38 minutes to 524.42 minutes.

## Bootstrap confidence intervals for other statistics

### Do people with no children under 14 spend more time resting? 

- Let's create a dataset (`DataFrame`) with children under 14 and time spent resting.

In [ ]:
cols = ['dur01', 'chh0014c']

colnames = {cols[0] : 'resttime',
            cols[1] : 'children'}

restchild_df = time_use_df[cols]

restchild_df = restchild_df.copy()

restchild_df.rename(columns = colnames, inplace=True)

restchild_df.head()

Create a new column `'Child_in_house'` that indicates if a child is in the house.

In [ ]:
restchild_df.loc[restchild_df['children'] > 0, 'Child_in_house'] = 'Yes'

restchild_df.loc[restchild_df['children'] == 0, 'Child_in_house'] = 'No'

restchild_df.head()

In [ ]:
mean_rest_times = restchild_df.groupby('Child_in_house')['resttime'].mean()

print(mean_rest_times)

print('\n Mean difference:', mean_rest_times[0] - mean_rest_times[1])

In [ ]:
restchild_df[['Child_in_house', 'resttime']].hist(by = 'Child_in_house', bins = 10, edgecolor = 'black', color = 'grey');

Create a function to create one bootstrap sample of the median difference.

In [ ]:
child = restchild_df['Child_in_house'] == 'Yes'

nochild = restchild_df['Child_in_house'] == 'No'

def bs_one_mean_diff():
    # sample body mass with replacement
    bssample = restchild_df.sample(frac = 1, replace =  True)
    
    # compute difference in medians on bootstrap sample
    bs_mean_diff = (bssample.loc[nochild, 'resttime'].mean() - 
                   bssample.loc[child, 'resttime'].mean())
    return bs_mean_diff

Run the function to compute one bootstrap mean difference.

In [ ]:
bs_one_mean_diff()

- Compute bootstrap distribution. 

In [ ]:
bootst_mean_diffs = []

for _ in range(5000):
    bootst_mean_diffs.append(bs_one_mean_diff())

In [ ]:
plt.hist(bootst_mean_diffs, color = 'grey', edgecolor = 'black');

A 95% confidence interval for the median difference is:

In [ ]:
np.percentile(bootst_mean_diffs, 2.5), np.percentile(bootst_mean_diffs, 97.5) 

The confidence interval does not contain 0, therefore there is evidence that the means are different.

## In-class exercise

Modify the code above to compute a 95% bootstrap confidence interval for the difference in means between Carnivora and Primates.  Does the confidence interval contain 0?

# Linear Regression

## Basic idea

- Linear regression is a useful technique for creating models to explain relationships between variables. 

- For use in multiple regression variables the dependent variable must be numeric, and have meaningful numeric values. 

- The independent variables can be interval or categorical variables.

## Example 1: perfect linear relationship between dependent and independent variables

### Data

In [ ]:
import pandas as pd
import numpy as np

data = {'depvar' : np.arange(start = 0, stop = 8, step =1),
       'indvar' : np.arange(start = 0, stop = 8, step =1)}

df = pd.DataFrame(data)

df

### Plot the data

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x = df['indvar'], y = df['depvar'])
plt.xlabel('indvar')
plt.ylabel('depvar');

- The scatter plot shows each pair of points (1, 1), (2, 2), etc. 

- `'indvar'` is sometimes called a predictor variable or covariate, and `'depvar'` is called the dependent variable.

- The dependent variable is predicted exactly by the independent variable.

### Compute the regression line

In [ ]:
from statsmodels.formula.api import ols

regmod = ols('depvar ~ indvar', data = df) # setup the model

- The code above uses the `ols` function from `statsmodels.formula.api` 

- The syntax in the function `ols` function `'depvar ~ indvar'` is a special syntax for describing statistical models.  

- The column name to the **left** of `~` specifies the dependent variable.

- The column name to the **right** of `~` specifies the independent variable(s).

In [ ]:
regmod_fit = regmod.fit() # estimate/fit the model 

After the model is setup then the `fit` function can be applied to the model.  This function computes the equation of the regression line.

In [ ]:
regmod_fit.params # get parameter estimates

- The estimates of the **y-intercept** and **slope** are labelled `Intercept` (1.11e-16 approx 0)  and `indvar` (1.00e+00 approx 1).

- This means the regression equation is: $$\texttt{depvar} = 0 + 1 \times \texttt{indvar}$$

## Example 2: another perfect linear relationship

In [ ]:
data = {'depvar' : np.arange(start = 0, stop = 8, step =1) + 2,
       'indvar' : np.arange(start = 0, stop = 8, step =1)}

df = pd.DataFrame(data)

df

In [ ]:
plt.scatter(x = df['indvar'], y = df['depvar'])
plt.xlabel('indvar')
plt.ylabel('depvar');

In [ ]:
regmod = ols('depvar ~ indvar', data = df) # setup the model

regmod_fit = regmod.fit() # estimate/fit the model 

regmod_fit.params # get parameter estimates

- The scatter plot is similar to Example 1 except that the values of the dependent variable has been increased by 2.

- The equation of the regression line for this data is:  $$\texttt{depvar} = 2 + 1 \times \texttt{indvar} $$

### Interpretation of y-intercept

- When the independent variable is 0 the dependent variable is 2.  This is the meaning of the **y-intercept** value of 2.

### Interpretation of slope

- For a one-unit change in the independent variable the dependent variable changes by 1 unit.

## Example 3: close to linear

- Examples 1 and 2 were perfect linear relationships.

- In this example we examine what happens if the relationship between the dependent and independent variables is almost linear.

In [ ]:
np.random.seed(11) #set the seed so that it's reproducible

data = {'depvar' : np.arange(start = 0, stop = 8, step =1) + 2,
        'indvar' : np.arange(start = 0, stop = 8, step =1) + np.random.uniform(low = 0, high = 2, size = 8)}

df = pd.DataFrame(data)

df

In [ ]:
plt.scatter(x = df['indvar'], y = df['depvar'])
plt.xlabel('indvar')
plt.ylabel('depvar');

In [ ]:
regmod = ols('depvar ~ indvar', data = df) # setup the model

regmod_fit = regmod.fit() # estimate/fit the model 

regmod_fit.params # get parameter estimates

So, now the relationship isn't perfectly linear, but close.  The equation of this regression line is:

$$\texttt{depvar} = 1.63 + 0.92 \times \texttt{indvar}$$

In [ ]:
import seaborn as sns

sns.regplot(x = 'indvar', y = 'depvar', data = df, ci = None);

The `regplot` function in the `seaborn` library will produce a scatter plot with the regression line.

The parameters of `regplot`

- `x` is the independent variable.
- `y` is the dependent variable.
- `ci = None` specifies no confidence interval for the regression line 

### Statistical summary of linear regression

In [ ]:
import warnings
warnings.filterwarnings('ignore') # turn off warnings

regsum = regmod_fit.summary()
regsum.tables[1]

- A statistical summary of the regression model is given above using the `summary` function in `statsmodels`.

- There are three summary tables, but we will only be interested in the second summary table---`regsum.tables[1]` (in this course).

What does the statistical summary represent?

- the column labelled `coef` are same values as `regmod_fit.params`.  In this case the average increase `depvar` for a one unit change in `indvar` is 0.9229.

- the `std err` column represents the standard deviation of the intercept and slope (we won't discuss in this course).

- the `t` column represents the value of the t-statistic (we won't discuss in this course).

- the column `P>|t|` represents the p-value corresponding to the null hypothesis that the intercept or slope are equal to 0.  If the value is small then this is evidence against the null hypothesis.  But if the p-value is large then there is no evidence against the null hypothesis.  

- Currently, how small a p-value should be to provide evidence against a null hypothesis is being debated.  Although, the traditional threshold is 0.05 (i.e., a p-value less than or equal to 0.05 is supposed to indicate evidence against the null hypothesis).

- In the example above the p-value for the slope is 0.  This indicates strong evidence that the *true* value of the slope is non-zero.

- The columns `[0.025   0.975]` form a plausible range of values for the y-intercept and slope (formally known as a 95% confidence interval). In this case the plausible values for the slope are 0.745 and 1.101.  If the plausible range includes 0 then this is usually interpreted as not providing evidence against the null hypothesis.  

## Predicted values and residuals

If the values of the independent variable are plugged into the regression equation then we obtain the fitted values.

$$\texttt{depvar} = 1.63 + 0.92 \times \texttt{indvar}$$

In [ ]:
df

The fitted value for the first row of `df` is:

In [ ]:
1.6254 + 0.9229*0.360539

- If the linear regression model is used on an independent variable that is not in the data set used to build the model then it's often referred to as a predicted value.

To extract the fitted values from a regression model use the `fittedvalues` function in `statsmodels`.

In [ ]:
regmod_fit.fittedvalues

The residual is the dependent variable minus the fitted value.  So, for the first row the residual is:

In [ ]:
2 - 1.9581414431

To extract the residuals from a regression model use the `resid` function in `statsmodels`.

In [ ]:
regmod_fit.resid

- If the linear regression model fits the data well then the residuals should be close to 0.

- One indication that the linear regression model fits well is to examine the **diagnostic** scatter plot of residuals versus fitted values.

- A plot that looks like a random scatter of points around 0 indicates that linear regression is an appropriate model.

- Since there are only 8 points it's hard to spot a pattern.

In [ ]:
plt.scatter(regmod_fit.fittedvalues, regmod_fit.resid)
plt.axhline(y = 0)
plt.xlabel('fitted values')
plt.ylabel('residuals')

## Accuracy of linear regression

- There are several measures of accuracy for linear regression.

- One popular measure is R-squared.

- It can be calculated from a fitted model regression model using the `rsquared` function in `statsmodels`.

- R-squared is always between 0 and 1.  
     + R-squared of 0 indicates a poor fit
     + R-squared of 1 indicates a perfect fit

In [ ]:
regmod_fit.rsquared

## Example 4: regression with two or more independent variables

It's possible to include more than one independent variable in a regression model.  

In [ ]:
np.random.seed(11) #set the seed so that it's reproducible

data = {'depvar' : np.arange(start = 0, stop = 8, step =1) + 2,
        'indvar1' : np.arange(start = 0, stop = 8, step =1) + np.random.uniform(low = 0, high = 2, size = 8),
        'indvar2' : np.arange(start = 0, stop = 8, step =1) + np.random.normal(loc = 0, scale = 1, size = 8)}

df = pd.DataFrame(data)

df

To include more than one independent variable in a regression model add it to the right side of syntax for describing statistical models.

`'depvar ~ indvar1 + indvar2'`

In [ ]:
regmod = ols('depvar ~ indvar1 + indvar2', data = df) # setup the model

Now `fit` `regmod`.

In [ ]:
regmod_fit = regmod.fit() # estimate/fit the model 

Extract the regression parameter estimates using `params`.

In [ ]:
regmod_fit.params # get parameter estimates

The estimated linear (multiple) linear regression model is:

$$\texttt{depvar} = 1.892482 + 0.549341 \times \texttt{indvar1} + 0.385197 \times \texttt{indvar2}$$

- The y-intercept value (1.892482) is the value of the dependent variable when `indvar1` and `indvar2` are both 0.

-  If `indvar1` changes by one-unit, `depvar` changes (on average) by 0.549341.

-  If `indvar2` changes by one-unit, `depvar` changes (on average) by 0.385197. 

### Statistical summary of a multiple regression model

In [ ]:
multregsummary = regmod_fit.summary()

multregsummary.tables[1]

The statistical summary indicates:

- the p-value for `indvar1` is 0.003 and the range of plausible values for the slope is 0.288 to 0.810.

- the p-value for `indvar2` is 0.010 and the range of plausible values for the slope is 0.137 to 0.634.  

- Since, both p-values are small there is evidence that slope is different from 0, and the range of plausible values does not include 0.

### Accuracy of multiple regression model

In [ ]:
plt.scatter(regmod_fit.fittedvalues, regmod_fit.resid)
plt.axhline(y = 0)
plt.xlabel('fitted values')
plt.ylabel('residuals')

In [ ]:
regmod_fit.rsquared

## What is the effect of recent immigration on asthma rates in Toronto neighbourhoods?

### Data

- Data from <http://torontohealthprofiles.ca> contains data on asthma and immigration for each neighbourhood in Toronto.

#### Sociodemographic data

- Read sociodemographic data from the excel file `1_socdem_neighb_2006-2.xls` with sheet name `socdem_2006` into `pandas` using `read_excel`.


In [ ]:
fname = '1_socdem_neighb_2006-2.xls'

sname = 'socdem_2006'

socdem_neighb = pd.read_excel(fname, sheet_name = sname, header = 10)

socdem_neighb.head()

Which column should we select to measure immigration?  There are several choices.

In [ ]:
socdem_neighb.columns

- Let's use `'% Recent immigrants-within 5 years'`.

- Later on we will want a few more sociodemographic columns, so let's create a new `DataFrame` and rename the columns.

In [ ]:
cols = ['Neighbourhood id', 'Neighbourhood Name', 
        'Median household income after-tax $ ‡', 
        '% Rented Dwellings', '% Unemployment rate *', 
        '% Recent immigrants-within 5 years', '% Visible minority']

socdem_neighb = socdem_neighb[cols]

colnames = {cols[0] : 'Neighbid',
            cols[1] : 'name',
            cols[2] : 'median_income',
            cols[3] : 'rented_dwell',
            cols[4] : 'unemployment',
            cols[5] : 'immigration5',
            cols[6] : 'vismin'}

socdem_neighb = socdem_neighb.copy()

socdem_neighb.rename(columns = colnames, inplace = True)

In [ ]:
socdem_neighb.head()

#### Asthma data

- Read data from the excel file `1_ahd_neighb_db_ast_hbp_mhv_copd_2007.xls` with sheet name `1_ahd_neighb_asthma_2007` into `pandas` using `read_excel`.

In [ ]:
fname = '1_ahd_neighb_db_ast_hbp_mhv_copd_2007.xls'
sname = '1_ahd_neighb_asthma_2007'

asthma_neighb = pd.read_excel(fname, sheet_name = sname, header = 11)
asthma_neighb.head()

- The asthma rates are broken down by sex and age.  

- Which rates should we choose?  

- For this example, let's select asthma rates for both sexes age 20 - 64.

In [ ]:
asthma_neighb.columns

In [ ]:
important_cols = asthma_neighb.columns[[0, 1, 11]]

In [ ]:
colnames = {important_cols[0]: 'Neighbid',
           important_cols[1] : 'name',
           important_cols[2] : 'asthma_pct'}

asthma_rates = asthma_neighb.copy()

asthma_rates = asthma_rates[important_cols]

asthma_rates.rename(columns = colnames, inplace=True)

asthma_rates.head()

### Merge Sociodemographic and asthma data

- To examine the relationship between asthma and immigration we will need to merge `socdem_neighb` and `asthma_neighb`.

- The `DataFrame`s can be merged on neighbourhood id.

In [ ]:
asthma_socdem = asthma_rates.merge(socdem_neighb, left_on='Neighbid', right_on='Neighbid')

asthma_socdem.head()

## A regression model of asthma on immigration

### Plot the data

- What are the dependent and independent variables?

- Does the relationship look linear?

In [ ]:
sns.regplot(y = 'asthma_pct', x = 'immigration5', data = asthma_socdem, ci = None);

### Fit the regression model

In [ ]:
reg_asthmaimm = ols('asthma_pct ~ immigration5', data = asthma_socdem) # setup the model

reg_asthmaimm_fit = regmod.fit() # estimate/fit the model 

### Statistical summary of the regression model

In [ ]:
reg_asthmaimm_summ = reg_asthmaimm_fit.summary()

reg_asthmaimm_summ.tables[1]

- The regression model is the equation:

$$\texttt{asthma_pct} = 11.8266  - 0.1597 \times \texttt{immigration5}$$

- The **slope** indicates that for a 1% **decrease** in immigration in the past 5 years the percentage of asthma cases in a neighbourhood decrease by 0.1597 (approximately 16%).

- The **y-intercept** indicates that when a neighbourhood has 0% immigration the past 5 years then asthma rates are 11%.  

- Are there any neighbourhoods with 0% immigration in the past 5 years?

In [ ]:
asthma_socdem['asthma_pct'].describe()

- The **y-intercept** in regression models often doesn't have a sensible interpretation, but is often mathematically important for regression to work well.

## Accuracy of the regression model 

In [ ]:
reg_asthmaimm_fit.rsquared

In [ ]:
plt.scatter(x = reg_asthmaimm_fit.fittedvalues , y = reg_asthmaimm_fit.resid)
plt.axhline(y = 0)
plt.xlabel('fitted values')
plt.ylabel('residuals')

## Training and testing a model

- Ideally we could assess the accuracy of this linear regression model on a new set of data (i.e., immigration rates in Toronto neighbourhoods in future years. 

- But, we don't have this data.

- One trick that data scientists use to fit the model on part of the data (training data) and use the remaining part (testing data) to test the accuracy.

- If we only have one data set then the training data can be obtained by randomly selecting rows from the data to fit the regression model, then the remaining rows can be used to test the accuracy of the model.

### Splitting a `pandas` `DataFrame` by `index`

In [ ]:
data = {'a' : [1, 2, 3, 4, 5]}

df = pd.DataFrame(data)

df.index

A Boolean array that is `True` if the index is `0` or `1` and `False` otherwise. 

In [ ]:
df.index.isin([0,1])

To create a Boolean series that is `False` if the index is `0` or `1` and `True` otherwise we can negate `df.index.isin([0,1])` this Boolean series using the `~` operator `~df.index.isin([0,1])`

In [ ]:
~df.index.isin([0,1])

In [ ]:
df[~df.index.isin([0,1])]

### Creating training and test data sets from a single dataset

### Step 1: 

- split the data into a training set with 75% of the rows.

- use the remaining 25% of the data for testing. 

In [ ]:
np.random.seed(11) # for reproducibility

# randomly select 75% of neighbourhoods
reg_df_train = asthma_socdem.sample(frac = 0.75, replace = False) 

# get index of training data
train_index = reg_df_train.index

- Exclude indicies from `reg_df_train` using `pandas` `isin` function. 

In [ ]:
# exclude rows in training to define test data
reg_df_test = asthma_socdem[~asthma_socdem.index.isin(train_index)]

print(asthma_socdem.shape)

print(reg_df_train.shape)

print(reg_df_test.shape)

### Step 2: Fit the regression model on the training data

In [ ]:
reg_train = ols('asthma_pct ~ immigration5', data = reg_df_train) # setup the model

reg_train_fit = reg_train.fit() # estimate/fit the model 

### Step 3: Compute predicted (fitted) values using training data 

In [ ]:
# use the model fit on the training data to predict asthma rates
# from the test set. 

predvalues_train = reg_train_fit.predict(reg_df_train['immigration5'])

### Step 4: Evaluate accuracy using root mean-squared error on training data

- Another measure of accuracy of regression models.

- Compares observed values of the dependent variable with the predicted values.

- It can be computed using the `rmse` function from `statsmodels`.

 Compute root mean-squared error for the training data.

In [ ]:
# compute the accuracy of these predictions

from statsmodels.tools.eval_measures import rmse

rmse(predvalues_train, reg_df_train['asthma_pct'])

- The observed asthma rates deviate from the predicted asthma rates in each neighbourhood by 1.4% in the training set.

- Is this an acceptable prediction error?

- Let's examine the accuracy of the linear regression model on the test set

- First compute the predicted values using the training data.

### Step 5: Evaluate the accuracy of regression model on the test data using root mean-squared error

- Compute predictions using test data

In [ ]:
predvalues_test = reg_train_fit.predict(reg_df_test['immigration5'])

- Now compute the root mean-squared error for the test data using the model fit on the training data.

In [ ]:
rmse(predvalues_test, reg_df_test['asthma_pct'])

## Conclusion of data analysis using linear regression model

- The regression model appears to provide an accurate fit the data, although both the scatter plot and plot of residuals indicate that there is a lot of variation in asthma rates in neighbourhoods that have between 10% - 20% immigration.

- As the percentage of immigrants to a neighbourhood within the last 5 years *decreases* so do the asthma rates. 

- Regression analyses on aggregated data such as neighbourhoods can be misleading when compared to non-aggregated data.

## Are there other sociaodemographic  factors in our data that might effect asthma rates?

- Let's create a matrix scatter plot using `pairplot` in the `seaborn` library.

In [ ]:
sns.pairplot(asthma_socdem)

- Let's consider median income

- Fit the regression model on all the data and produce a statistical summary.

In [ ]:
reg_asthmaimmunem = ols('asthma_pct ~ immigration5 + median_income', data = asthma_socdem) # setup the model

reg_asthmaimmunem_fit = reg_asthmaimmunem.fit() # estimate/fit the model 

reg_asthmaimmunem_summ = reg_asthmaimmunem_fit.summary()

reg_asthmaimmunem_summ.tables[1]

- Notice that the slope estimate for `median_income` is 0, but so is the p-value and range of plausible values.

- That seems odd.  What's happening?

- We can see from the scatter plot that `immigration5` and `median_income` have a linear relationship.  When two variables have a linear relationship multiple linear regression models produce strange results.

- It's important that independent variables in a multiple regression model are unrelated.

# Review of Comparing two samples

## Comparing plant fertilizers

- A gardener wanted to discover whether a change in fertilizer mixture applied to her tomato plants would result in improved yield.

- She had 11 plants set out in a single row:
   + 5 were given standard fertilizer mixture A
   + 6 were given a supposedly improved mixture B

- The A's and B's were randomly applied to positions along the row to give the following data:

|                |  |   |   |   |   |   |   |   |   |   |   |
|----------------|---|---|---|---|---|---|---|---|---|---|---|
|Position in row | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10| 11 |
|Fertilizer      | A | A | B | B | A | B |B  |B  | A | A | B  |
|Pounds of tomatoes | 29.2 | 11.4 | 26.6 | 23.7 | 25.3 | 28.5 | 14.2 | 17.9 | 16.5 | 21.1 | 24.3 |


- The random arrangement was arrived at by taking 11 playing cards, 5 marked A, and 6 marked B.

- Thoroughly shuffling the cards once the gardener arrived at the arrangement above. 


In [ ]:
import pandas as pd

fertilizer = ['A', 'A', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'B']

tomatoes = [29.2, 11.4, 26.6, 23.7, 25.3, 28.5, 14.2, 17.9, 16.5, 21.1, 24.3]

data = {'fert' : fertilizer,
        'tomatoes' : tomatoes}

plant_df = pd.DataFrame(data)

The (observed) mean difference is computed below.

In [ ]:
# mean of plants assigned fert A
mean_A_obs = plant_df.loc[plant_df['fert'] == 'A', 'tomatoes'].mean() 

# mean of plants assigned fert B
mean_B_obs = plant_df.loc[plant_df['fert'] == 'B', 'tomatoes'].mean()

# mean difference
obs_diff =  mean_B_obs - mean_A_obs

obs_diff

- Since the assignment of fertilizers to plants is random it could have happened another way.

- We can use the `pandas` `sample` function with `frac` and `replace` parameters to simulate these other potential assignments of fertilizers to plants.

    + `frac = 1` - specifies the fraction of rows to return (1 means return all the rows)
    + `replace = False` - specifies to sample without replacement

In [ ]:
plant_df['fert'].sample(frac=1, replace = False)

- Notice that the index is out of order.

- We are going to want to have an ordered index later on.  

- To do this we can use the `pandas` function `reset_index(drop = True)`. 

- `drop = True` indicates that we don't want to keep the previous index. 

In [ ]:
plant_df['fert'].sample(frac=1, replace = False).reset_index(drop = True)

## The Logic of Hypothesis Testing

### 1. Hypotheses

Two claims:

1. There is no difference in the mean weight between fertilizers A and B.  This is called the null hypothesis.

2. There is no difference in the mean weight between fertilizers A and B.  This is called the alternative hypothesis.

### 2. Test statistic

The test statistic is a number, calculated from the data, that captures what we're interested in.

What would be a useful test statistic for this study?

### 3. Simulate what the null hypothesis predicts will happen

- If the null hypothesis is true then the weight of tomatoes for each plant will be the same regardless of how they are labeled. That means we can randomly shuffle the labels and the mean difference should be close to 0.

Assume that there is no difference in mean weight between A and B (i.e., the null hypothesis is true).  Now, consider the following **thought experiment (we don't actually do this, this is a model for the data)**:

- Imagine we have 5 playing cards labelled `A` and 6 cards labelled `B`.


- Shuffle the cards ...


- Assign the cards to the 11 plants then calculate the mean weight difference between `A` and `B`.  This is one simulated value of the test statistic. 


- Shuffle the cards again ...


- Assign the cards to the 11 plants then calculate the mean weight difference between `A` and `B`.  This is second simulated value of the test statistic. 


- Continue shuffling, assigning , and computing the mean difference.

## Simulating what the null hypothesis predicts

Let's assume the null hypothesis is true and simulate what the null hypothesis predicts.

### Step 1

Randomly shuffle the assignment of fertilizers to plants. 

In [ ]:
fert_sim = plant_df['fert'].sample(frac=1, replace = False).reset_index(drop = True)

### Step 2

Compute the mean difference between fertilizers A and B.

In [ ]:
mean_A = plant_df.loc[fert_sim == 'A', 'tomatoes'].mean()

mean_B = plant_df.loc[fert_sim == 'B', 'tomatoes'].mean()

sim_diff = mean_B - mean_A

sim_diff

### Step 3

Repeat Steps 1 and 2 a large number of times (e.g., 5000) to get the distribution of mean differences. 

In [ ]:
simulated_diffs = []

for _ in range(5000):
    fert_sim = plant_df['fert'].sample(frac=1, replace = False).reset_index(drop = True)
    mean_A = plant_df.loc[fert_sim == 'A', 'tomatoes'].mean()
    mean_B = plant_df.loc[fert_sim == 'B', 'tomatoes'].mean()
    sim_diff = mean_B - mean_A
    simulated_diffs.append(sim_diff)
    

In [ ]:
import matplotlib.pyplot as plt

plt.hist(simulated_diffs, edgecolor = 'black', color = 'lightgrey')
#plt.vlines(x = obs_diff, ymin = 0, ymax = 1000, color = 'black', linewidth = 5);

- The histogram above shows the randomization distribution with the observed difference as the black line.

- What proportion of the simulated differences are larger than the observed mean difference of 1.83?  This is known as the **p-value**.

In [ ]:
simulated_diffs_df = pd.DataFrame({'sim_diffs' : simulated_diffs})

numgreater = (simulated_diffs_df['sim_diffs'] >= obs_diff).sum()

print('The number of simulated differences greater than the observed difference is:', numgreater)

pvalue = numgreater / 5000

print('The p-value is:', pvalue)

### Step 4

Interpret the results.  

Assuming that there is no difference in the mean tomato plant weights between A and B, 31% of simulations had as large or larger value than the observed mean difference of 1.83.  Therefore, there is little reason to doubt the null hypothesis that one fertilizer is better than another.